In [2]:
import warnings
warnings.filterwarnings(action="ignore")
import os
import ast
import pandas as pd
import numpy as np
from matplotlib import pyplot
import matplotlib.patches as mpatches
import seaborn as sn
from tqdm.std import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [3]:
class Config:
    data_dir = '../data/'
    working_dir = '../src/'
    models_dir = '../models/'
    submissions_dir = '../submissions'

In [4]:
train = pd.read_csv("Train.csv")
metadata = pd.read_csv("metadata.csv")
test = pd.read_csv("Test.csv")
ss = pd.read_csv("SampleSubmission.csv")

In [5]:
print(metadata.isna().sum())

ID                         0
RegistrationDate           0
Deposit                    0
UpsellDate             36370
AccessoryRate              0
PaymentMethod              0
rateTypeEntity             0
RatePerUnit                0
DaysOnDeposit              0
MainApplicantGender        0
Age                     6939
Region                  1934
Town                       0
Occupation                 0
SupplierName               0
Term                       0
TotalContractValue         0
ExpectedTermDate           0
FirstPaymentDate           0
LastPaymentDate            0
dtype: int64


In [6]:
metadata[metadata.isna().Region].head(5)

,ID,RegistrationDate,Deposit,UpsellDate,AccessoryRate,PaymentMethod,rateTypeEntity,RatePerUnit,DaysOnDeposit,MainApplicantGender,Age,Region,Town,Occupation,SupplierName,Term,TotalContractValue,ExpectedTermDate,FirstPaymentDate,LastPaymentDate
3,ID_HXBJFHB,2015-11-25 00:00:00,2000,NaN,0.0,FINANCED,DAILY,35,7,Female,43.0,NaN,UNKNOWN,Teacher,d_light,364,14740.0,2016-11-23 00:00:00,2015-11-25 13:25:57,2017-05-22 16:46:54
17,ID_AYIBQUP,2015-12-12 00:00:00,2000,NaN,0.0,FINANCED,DAILY,35,7,Male,44.0,NaN,UNKNOWN,Other,d_light,364,14740.0,2016-12-10 00:00:00,2015-12-12 07:00:38,2020-09-02 20:30:53
51,ID_35MNQL1,2015-12-06 00:00:00,2000,NaN,0.0,FINANCED,DAILY,35,7,Male,42.0,NaN,UNKNOWN,Farmer,d_light,364,14740.0,2016-12-04 00:00:00,2015-12-07 06:21:45,2017-11-18 11:07:25
84,ID_RWA0Q3U,2017-07-12 14:44:48,2000,NaN,0.0,FINANCED,DAILY,40,7,Female,52.0,NaN,UNKNOWN,Government Employee,d_light,364,16560.0,2018-07-11 14:44:48,2017-07-12 14:45:19,2020-09-02 20:31:12
85,ID_QWI7WWN,2015-11-24 00:00:00,2000,NaN,0.0,FINANCED,DAILY,35,7,Male,40.0,NaN,UNKNOWN,Other,d_light,364,14740.0,2016-11-22 00:00:00,2015-11-24 14:02:51,2016-11-25 14:58:45


In [7]:
metadata.Region.value_counts()

Nyanza                6998
North Rift            5344
Nairobi Region        5056
South Rift            4759
Coast Region          4671
Western               4508
Mount Kenya Region    4073
Name: Region, dtype: int64

In [8]:
pd.merge(left=metadata[metadata.isna().Region], right=test, on="ID").head(2)
#test data also contains empty values in region therefore we need to treat as a category.

,ID,RegistrationDate,Deposit,UpsellDate,AccessoryRate,PaymentMethod,rateTypeEntity,RatePerUnit,DaysOnDeposit,MainApplicantGender,Age,Region,Town,Occupation,SupplierName,Term,TotalContractValue,ExpectedTermDate,FirstPaymentDate,LastPaymentDate,TransactionDates,PaymentsHistory
0,ID_UOT1MF3,2017-07-13 08:56:05,2000,NaN,0.0,FINANCED,DAILY,40,7,Male,54.0,NaN,UNKNOWN,Business,d_light,364,16560.0,2018-07-12 08:56:05,2017-07-13 08:56:23,2020-09-02 20:31:28,"['07-2017', '08-2017', '09-2017', '10-2017', '11-2017', '12-2017', '01-2018', '02-2018', '03-2018', '04-2018', '05-2018', '06-2018', '07-2018', '08-2018', '10-2018', '11-2018', '12-2018', '01-2019', '02-2019', '03-2019', '04-2019', '07-2019']","[2720.0, 1080.0, 1200.0, 1000.0, 960.0, 1160.0, 1120.0, 600.0, 1040.0, 840.0, 680.0, 640.0, 680.0, 520.0, 80.0, 80.0, 40.0, 40.0, 240.0, 40.0, 80.0, 40.0]"
1,ID_290EB8N,2017-07-05 09:06:30,2000,NaN,0.0,FINANCED,DAILY,40,7,Male,31.0,NaN,UNKNOWN,Labourer,d_light,364,16560.0,2018-07-04 09:06:30,2017-07-05 08:54:37,2020-09-02 20:35:26,"['07-2017', '08-2017', '12-2017', '01-2018', '02-2018', '03-2018', '04-2018', '05-2018', '06-2018', '07-2018', '08-2018', '09-2018', '10-2018']","[2710.0, 120.0, 200.0, 300.0, 50.0, 250.0, 550.0, 100.0, 500.0, 350.0, 350.0, 950.0, 200.0]"


In [9]:
metadata.Region.fillna(value="Other", inplace=True)

In [10]:
metadata.UpsellDate.fillna(value=0, inplace=True)

In [11]:
pd.merge(left=metadata[metadata.isna().Age], right=test, on="ID").head(2)
##now just using mean here to remove 0 in metadata.


,ID,RegistrationDate,Deposit,UpsellDate,AccessoryRate,PaymentMethod,rateTypeEntity,RatePerUnit,DaysOnDeposit,MainApplicantGender,Age,Region,Town,Occupation,SupplierName,Term,TotalContractValue,ExpectedTermDate,FirstPaymentDate,LastPaymentDate,TransactionDates,PaymentsHistory
0,ID_VJ80SX2,2015-12-14 00:00:00,2000,0,0.0,FINANCED,DAILY,35,7,Female,NaN,Mount Kenya Region,Embu,Business,d_light,364,14740.0,2016-12-12 00:00:00,2015-12-14 14:20:26,2016-12-12 11:59:20,"['12-2015', '01-2016', '02-2016', '03-2016', '04-2016', '05-2016', '06-2016']","[3000.0, 850.0, 750.0, 1500.0, 650.0, 1250.0, 1000.0]"
1,ID_ZLW8XIB,2015-12-05 00:00:00,2000,0,0.0,FINANCED,DAILY,35,7,Male,NaN,Mount Kenya Region,Embu,Farmer,d_light,364,14740.0,2016-12-03 00:00:00,2015-12-05 15:28:45,2017-04-01 19:19:19,"['12-2015', '01-2016', '02-2016', '03-2016', '04-2016', '05-2016', '06-2016', '07-2016', '08-2016', '09-2016', '10-2016']","[3200.0, 735.0, 1320.0, 1000.0, 1170.0, 866.0, 730.0, 100.0, 100.0, 450.0, 700.0]"


In [12]:
metadata.Age.fillna(value=round(metadata.Age.mean()), inplace=True)

In [13]:
merged = pd.merge(left=metadata, right=pd.concat(objs=[train, test]).fillna(value=0), on="ID")
merged.head(5)

,ID,RegistrationDate,Deposit,UpsellDate,AccessoryRate,PaymentMethod,rateTypeEntity,RatePerUnit,DaysOnDeposit,MainApplicantGender,Age,Region,Town,Occupation,SupplierName,Term,TotalContractValue,ExpectedTermDate,FirstPaymentDate,LastPaymentDate,TransactionDates,PaymentsHistory,m1,m2,m3,m4,m5,m6
0,ID_K00S4N4,2015-12-10 00:00:00,2000,0,0.0,FINANCED,DAILY,35,7,Male,41.0,Mount Kenya Region,Embu,Other,d_light,364,14740.0,2016-12-08 00:00:00,2015-12-10 09:52:35,2016-10-23 04:52:30,"['12-2015', '01-2016', '02-2016', '03-2016', '04-2016']","[3050.0, 1050.0, 910.0, 1050.0, 1050.0]",1225.0,1050.0,1190.0,525.0,1750.0,1890.0
1,ID_6L67PAA,2015-12-09 00:00:00,2000,0,0.0,FINANCED,DAILY,35,7,Male,33.0,Coast Region,Kilifi,Other,d_light,364,14740.0,2016-12-07 00:00:00,2015-12-09 13:14:03,2020-05-24 15:32:18,"['12-2015', '01-2016', '02-2016', '03-2016', '05-2016', '07-2016']","[4000.0, 1050.0, 1050.0, 1050.0, 1050.0, 400.0]",0.0,0.0,0.0,0.0,0.0,0.0
2,ID_102CV85,2015-12-18 00:00:00,2000,2018-03-29 10:14:58,35.0,FINANCED,DAILY,35,7,Female,48.0,Nairobi Region,Makueni,Business,d_light,392,29480.0,2017-01-13 00:00:00,2015-12-18 06:22:34,2017-02-01 15:23:44,"['12-2015', '01-2016', '02-2016', '03-2016', '04-2016', '05-2016', '06-2016', '07-2016', '08-2016']","[4245.0, 980.0, 735.0, 735.0, 1470.0, 735.0, 980.0, 915.0, 735.0]",980.0,980.0,1225.0,980.0,935.0,355.0
3,ID_HXBJFHB,2015-11-25 00:00:00,2000,0,0.0,FINANCED,DAILY,35,7,Female,43.0,Other,UNKNOWN,Teacher,d_light,364,14740.0,2016-11-23 00:00:00,2015-11-25 13:25:57,2017-05-22 16:46:54,"['11-2015', '12-2015', '01-2016', '02-2016', '03-2016', '04-2016', '05-2016', '06-2016', '07-2016', '08-2016', '10-2016']","[2245.0, 980.0, 980.0, 1225.0, 980.0, 980.0, 980.0, 1225.0, 735.0, 490.0, 250.0]",250.0,1000.0,250.0,500.0,560.0,1150.0
4,ID_3K9VZ5J,2015-12-02 00:00:00,2000,0,0.0,FINANCED,DAILY,35,7,Female,56.0,Mount Kenya Region,Kirinyaga,Other,d_light,364,14740.0,2016-11-30 00:00:00,2015-12-05 10:34:32,2017-05-12 16:50:52,"['12-2015', '01-2016', '02-2016', '03-2016', '04-2016', '05-2016', '06-2016', '07-2016', '08-2016', '09-2016', '10-2016', '11-2016']","[2750.0, 1000.0, 750.0, 1000.0, 955.0, 880.0, 280.0, 665.0, 770.0, 420.0, 525.0, 735.0]",630.0,805.0,700.0,855.0,245.0,775.0


In [14]:
merged.drop(["FirstPaymentDate"],axis=1,inplace=True)

In [15]:
def diff(a,b):
    start=b.split('-')
    end=a.split('-')
    return int(end[1])-int(start[1])+12*(int(end[0])-int(start[0]))

In [16]:
#merged['month']=merged['RegistrationDate']
#merged['year']=merged['RegistrationDate']
merged['time']=merged['RegistrationDate']
merged['PaidAmount']=merged['TotalContractValue']
merged['PeriodDiff']=merged['TotalContractValue']
for i in tqdm(range(0,merged.shape[0]),desc='Row'):
    year=int(merged.RegistrationDate.iloc[i].split('-')[0])
    month=int(merged.RegistrationDate.iloc[i].split('-')[1])
    time=np.ceil(merged.loc[i,'Term']/30)
    merged.loc[i,'time']=time
    if merged.UpsellDate.iloc[i]!=0:
        merged.loc[i,'UpsellDate']=diff(merged.UpsellDate.iloc[i],merged.RegistrationDate.iloc[i])/time
    merged.loc[i,'LastPaymentDate']=diff(merged.LastPaymentDate.iloc[i],merged.RegistrationDate.iloc[i])/time
    merged.loc[i,'ExpectedTermDate']=diff(merged.ExpectedTermDate.iloc[i],merged.RegistrationDate.iloc[i])/time
    merged.loc[i,'m1']=merged.loc[i,'m1']/merged.loc[i,'TotalContractValue']
    merged.loc[i,'m2']=merged.loc[i,'m2']/merged.loc[i,'TotalContractValue']
    merged.loc[i,'m3']=merged.loc[i,'m3']/merged.loc[i,'TotalContractValue']
    merged.loc[i,'m4']=merged.loc[i,'m4']/merged.loc[i,'TotalContractValue']
    merged.loc[i,'m5']=merged.loc[i,'m5']/merged.loc[i,'TotalContractValue']
    merged.loc[i,'m6']=merged.loc[i,'m6']/merged.loc[i,'TotalContractValue']
    dates=[]
    for j in merged.loc[i,'TransactionDates'].split("'"):
        if '-' in j:
            date=j.split('-')
            dates.append((int(date[0])-month+12*(int(date[1])-year))/time)
    merged.at[i,'TransactionDates']=dates
    history=[]
    for j in merged.loc[i,'PaymentsHistory'].split("[")[1].split("]")[0].split(","):
        if '.' in j:
            history.append(float(j)/merged.loc[i,'TotalContractValue'])
    merged.at[i,'PaymentsHistory']=history
    merged.loc[i,'PaidAmount']=sum(history)
    merged.loc[i,'PeriodDiff']=merged.loc[i,'LastPaymentDate']-dates[-1]
merged.drop(["RegistrationDate","Term"],axis=1,inplace=True)

Row: 100%|██████████| 37343/37343 [04:13<00:00, 147.19it/s]


In [17]:
merged.tail(2)

,ID,Deposit,UpsellDate,AccessoryRate,PaymentMethod,rateTypeEntity,RatePerUnit,DaysOnDeposit,MainApplicantGender,Age,Region,Town,Occupation,SupplierName,TotalContractValue,ExpectedTermDate,LastPaymentDate,TransactionDates,PaymentsHistory,m1,m2,m3,m4,m5,m6,time,PaidAmount,PeriodDiff
37341,ID_GHHAQ9D,2400,0,0.0,FINANCED,DAILY,50,3,Male,27.0,Other,UNKNOWN,Labourer,d_light,14400.0,1.0,1.75,"[0.0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0]","[0.18194444444444444, 0.09027777777777778, 0.08333333333333333, 0.07291666666666667, 0.0763888888888889, 0.04513888888888889, 0.010416666666666666, 0.0038194444444444443, 0.021875]",0.034722,0.059028,0.045139,0.048611,0.041667,0.048611,8.0,0.586111,0.75
37342,ID_PAI1FJK,2400,0,0.0,FINANCED,DAILY,50,3,Male,36.0,Coast Region,Mombasa,Farmer,d_light,14400.0,1.0,1.875,"[0.0, 0.125, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0, 1.125]","[0.1701388888888889, 0.003472222222222222, 0.003472222222222222, 0.09375, 0.03125, 0.07291666666666667, 0.04861111111111111, 0.0798611111111111, 0.05277777777777778]",0.100694,0.055556,0.017361,0.003472,0.010417,0.045139,8.0,0.556250,0.75


In [18]:
from sklearn.preprocessing import LabelEncoder
lb_make = LabelEncoder()
category_list=['PaymentMethod','rateTypeEntity','MainApplicantGender','Region','Town','Occupation','SupplierName']
for s in category_list:
    merged[s] = lb_make.fit_transform(merged[s])
    print(lb_make.transform(lb_make.classes_))
    print(lb_make.classes_)
merged.head(2)

[0]
['FINANCED']
[0 1 2]
['DAILY' 'MONTHLY' 'WEEKLY']
[0 1]
['Female' 'Male']
[0 1 2 3 4 5 6 7]
['Coast Region' 'Mount Kenya Region' 'Nairobi Region' 'North Rift'
 'Nyanza' 'Other' 'South Rift' 'Western']
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47]
['Baringo' 'Bomet' 'Bungoma' 'Busia' 'Elgeyo/Marakwet' 'Embu' 'Garissa'
 'Homa Bay' 'Isiolo' 'Kajiado' 'Kakamega' 'Kericho' 'Kiambu' 'Kilifi'
 'Kirinyaga' 'Kisii' 'Kisumu' 'Kitui' 'Kwale' 'Laikipia' 'Lamu' 'Machakos'
 'Makueni' 'Mandera' 'Marsabit' 'Meru' 'Migori' 'Mombasa' 'Muranga'
 'Nairobi City' 'Nakuru' 'Nandi' 'Narok' 'Nyamira' 'Nyandarua' 'Nyeri'
 'Samburu' 'Siaya' 'Taita/Taveta' 'Tana River' 'Tharaka-Nithi'
 'Trans Nzoia' 'Turkana' 'UNKNOWN' 'Uasin Gishu' 'Vihiga' 'Wajir'
 'West Pokot']
[0 1 2 3 4 5 6]
['Business' 'Driver/Motorbike Rider' 'Farmer' 'Government Employee'
 'Labourer' 'Other' 'Teacher']
[0]
['d_light']


,ID,Deposit,UpsellDate,AccessoryRate,PaymentMethod,rateTypeEntity,RatePerUnit,DaysOnDeposit,MainApplicantGender,Age,Region,Town,Occupation,SupplierName,TotalContractValue,ExpectedTermDate,LastPaymentDate,TransactionDates,PaymentsHistory,m1,m2,m3,m4,m5,m6,time,PaidAmount,PeriodDiff
0,ID_K00S4N4,2000,0,0.0,0,0,35,7,1,41.0,1,5,5,0,14740.0,0.923077,0.769231,"[0.0, 0.07692307692307693, 0.15384615384615385, 0.23076923076923078, 0.3076923076923077]","[0.20691994572591588, 0.07123473541383989, 0.06173677069199457, 0.07123473541383989, 0.07123473541383989]",0.083107,0.071235,0.080733,0.035617,0.118725,0.128223,13.0,0.482361,0.461538
1,ID_6L67PAA,2000,0,0.0,0,0,35,7,1,33.0,0,13,5,0,14740.0,0.923077,4.076923,"[0.0, 0.07692307692307693, 0.15384615384615385, 0.23076923076923078, 0.38461538461538464, 0.5384615384615384]","[0.27137042062415195, 0.07123473541383989, 0.07123473541383989, 0.07123473541383989, 0.07123473541383989, 0.027137042062415198]",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,13.0,0.583446,3.538462


In [19]:
## As these two variable has only one class.
merged.drop(['PaymentMethod','SupplierName'],axis=1,inplace=True)

In [20]:
merged.head(5)

,ID,Deposit,UpsellDate,AccessoryRate,rateTypeEntity,RatePerUnit,DaysOnDeposit,MainApplicantGender,Age,Region,Town,Occupation,TotalContractValue,ExpectedTermDate,LastPaymentDate,TransactionDates,PaymentsHistory,m1,m2,m3,m4,m5,m6,time,PaidAmount,PeriodDiff
0,ID_K00S4N4,2000,0,0.0,0,35,7,1,41.0,1,5,5,14740.0,0.923077,0.769231,"[0.0, 0.07692307692307693, 0.15384615384615385, 0.23076923076923078, 0.3076923076923077]","[0.20691994572591588, 0.07123473541383989, 0.06173677069199457, 0.07123473541383989, 0.07123473541383989]",0.083107,0.071235,0.080733,0.035617,0.118725,0.128223,13.0,0.482361,0.461538
1,ID_6L67PAA,2000,0,0.0,0,35,7,1,33.0,0,13,5,14740.0,0.923077,4.076923,"[0.0, 0.07692307692307693, 0.15384615384615385, 0.23076923076923078, 0.38461538461538464, 0.5384615384615384]","[0.27137042062415195, 0.07123473541383989, 0.07123473541383989, 0.07123473541383989, 0.07123473541383989, 0.027137042062415198]",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,13.0,0.583446,3.538462
2,ID_102CV85,2000,1.928571,35.0,0,35,7,0,48.0,2,22,0,29480.0,0.928571,1.0,"[0.0, 0.07142857142857142, 0.14285714285714285, 0.21428571428571427, 0.2857142857142857, 0.35714285714285715, 0.42857142857142855, 0.5, 0.5714285714285714]","[0.14399592944369063, 0.03324287652645862, 0.024932157394843963, 0.024932157394843963, 0.049864314789687926, 0.024932157394843963, 0.03324287652645862, 0.031037991858887382, 0.024932157394843963]",0.033243,0.033243,0.041554,0.033243,0.031716,0.012042,14.0,0.391113,0.428571
3,ID_HXBJFHB,2000,0,0.0,0,35,7,0,43.0,5,43,6,14740.0,0.923077,1.384615,"[0.0, 0.07692307692307693, 0.15384615384615385, 0.23076923076923078, 0.3076923076923077, 0.38461538461538464, 0.46153846153846156, 0.5384615384615384, 0.6153846153846154, 0.6923076923076923, 0.8461538461538461]","[0.1523066485753053, 0.06648575305291723, 0.06648575305291723, 0.08310719131614654, 0.06648575305291723, 0.06648575305291723, 0.06648575305291723, 0.08310719131614654, 0.049864314789687926, 0.03324287652645862, 0.016960651289009497]",0.016961,0.067843,0.016961,0.033921,0.037992,0.078019,13.0,0.751018,0.538462
4,ID_3K9VZ5J,2000,0,0.0,0,35,7,0,56.0,1,14,5,14740.0,0.846154,1.307692,"[0.0, 0.07692307692307693, 0.15384615384615385, 0.23076923076923078, 0.3076923076923077, 0.38461538461538464, 0.46153846153846156, 0.5384615384615384, 0.6153846153846154, 0.6923076923076923, 0.7692307692307693, 0.8461538461538461]","[0.1865671641791045, 0.06784260515603799, 0.05088195386702849, 0.06784260515603799, 0.06478968792401628, 0.05970149253731343, 0.018995929443690638, 0.04511533242876527, 0.05223880597014925, 0.028493894165535955, 0.03561736770691994, 0.049864314789687926]",0.042741,0.054613,0.047490,0.058005,0.016621,0.052578,13.0,0.727951,0.461538


In [21]:
merged.to_csv("Final.csv",index=False)

##Advance:: Can use clustering/estimation to get age.
##Advance:: Care for Days on deposit(Some finance has to be read)
##Advance:: Care for deposit in paid amount.

In [22]:
from tsfresh import extract_features
df_final=pd.DataFrame(columns=["time","money","ID"])
for k in tqdm(range(0,merged.shape[0])):
    df_temp=pd.DataFrame({"time":merged.loc[k,"TransactionDates"],"money":merged.loc[k,"PaymentsHistory"]})
    df_temp['ID']=merged.loc[k,"ID"]
    df_final=pd.concat([df_final, df_temp], ignore_index=True)
print(df_final.shape)

100%|██████████| 37343/37343 [02:30<00:00, 248.62it/s]

(607851, 3)


In [23]:
new_features=extract_features(df_final,column_id='ID',column_sort="time").dropna(axis=1)

Feature Extraction: 100%|██████████| 20/20 [07:29<00:00, 22.49s/it]


In [24]:
new_features.head()

,money__variance_larger_than_standard_deviation,money__has_duplicate_max,money__has_duplicate_min,money__has_duplicate,money__sum_values,money__abs_energy,money__mean_abs_change,money__mean_change,money__mean_second_derivative_central,money__median,money__mean,money__length,money__standard_deviation,money__variation_coefficient,money__variance,money__skewness,money__kurtosis,money__root_mean_square,money__absolute_sum_of_changes,money__longest_strike_below_mean,money__longest_strike_above_mean,money__count_above_mean,money__count_below_mean,money__last_location_of_maximum,money__first_location_of_maximum,money__last_location_of_minimum,money__first_location_of_minimum,money__percentage_of_reoccurring_values_to_all_values,money__percentage_of_reoccurring_datapoints_to_all_datapoints,money__sum_of_reoccurring_values,money__sum_of_reoccurring_data_points,money__ratio_value_number_to_time_series_length,money__maximum,money__minimum,money__time_reversal_asymmetry_statistic__lag_1,money__time_reversal_asymmetry_statistic__lag_2,money__time_reversal_asymmetry_statistic__lag_3,money__c3__lag_1,money__c3__lag_2,money__c3__lag_3,money__cid_ce__normalize_True,money__cid_ce__normalize_False,money__symmetry_looking__r_0.0,money__symmetry_looking__r_0.05,money__symmetry_looking__r_0.1,money__symmetry_looking__r_0.15000000000000002,money__symmetry_looking__r_0.2,money__symmetry_looking__r_0.25,money__symmetry_looking__r_0.30000000000000004,money__symmetry_looking__r_0.35000000000000003,money__symmetry_looking__r_0.4,money__symmetry_looking__r_0.45,money__symmetry_looking__r_0.5,money__symmetry_looking__r_0.55,money__symmetry_looking__r_0.6000000000000001,money__symmetry_looking__r_0.65,money__symmetry_looking__r_0.7000000000000001,money__symmetry_looking__r_0.75,money__symmetry_looking__r_0.8,money__symmetry_looking__r_0.8500000000000001,money__symmetry_looking__r_0.9,money__symmetry_looking__r_0.9500000000000001,money__large_standard_deviation__r_0.05,money__large_standard_deviation__r_0.1,money__large_standard_deviation__r_0.15000000000000002,money__large_standard_deviation__r_0.2,money__large_standard_deviation__r_0.25,money__large_standard_deviation__r_0.30000000000000004,money__large_standard_deviation__r_0.35000000000000003,money__large_standard_deviation__r_0.4,money__large_standard_deviation__r_0.45,money__large_standard_deviation__r_0.5,money__large_standard_deviation__r_0.55,money__large_standard_deviation__r_0.6000000000000001,money__large_standard_deviation__r_0.65,money__large_standard_deviation__r_0.7000000000000001,money__large_standard_deviation__r_0.75,money__large_standard_deviation__r_0.8,money__large_standard_deviation__r_0.8500000000000001,money__large_standard_deviation__r_0.9,money__large_standard_deviation__r_0.9500000000000001,money__quantile__q_0.1,money__quantile__q_0.2,money__quantile__q_0.3,money__quantile__q_0.4,money__quantile__q_0.6,money__quantile__q_0.7,money__quantile__q_0.8,money__quantile__q_0.9,money__autocorrelation__lag_0,money__autocorrelation__lag_1,money__autocorrelation__lag_2,money__autocorrelation__lag_3,money__autocorrelation__lag_4,"money__agg_autocorrelation__f_agg_""mean""__maxlag_40","money__agg_autocorrelation__f_agg_""median""__maxlag_40","money__agg_autocorrelation__f_agg_""var""__maxlag_40",money__partial_autocorrelation__lag_0,money__partial_autocorrelation__lag_1,money__number_cwt_peaks__n_1,money__number_cwt_peaks__n_5,money__number_peaks__n_1,money__number_peaks__n_3,money__number_peaks__n_5,money__number_peaks__n_10,money__number_peaks__n_50,money__binned_entropy__max_bins_10,money__index_mass_quantile__q_0.1,money__index_mass_quantile__q_0.2,money__index_mass_quantile__q_0.3,money__index_mass_quantile__q_0.4,money__index_mass_quantile__q_0.6,money__index_mass_quantile__q_0.7,money__index_mass_quantile__q_0.8,money__index_mass_quantile__q_0.9,"money__cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)","money__cwt_coefficients__coeff_0__w_5__widths_(2, 5, 10, 20)","money__cwt_coef

In [25]:
new_features.to_csv("Final2.csv")

In [26]:
data1=pd.read_csv("Final.csv")
data2=pd.read_csv("Final2.csv")

In [27]:
data2.rename(columns={ data2.columns[0]: "ID" }, inplace = True)  

data2.head()

,ID,money__variance_larger_than_standard_deviation,money__has_duplicate_max,money__has_duplicate_min,money__has_duplicate,money__sum_values,money__abs_energy,money__mean_abs_change,money__mean_change,money__mean_second_derivative_central,money__median,money__mean,money__length,money__standard_deviation,money__variation_coefficient,money__variance,money__skewness,money__kurtosis,money__root_mean_square,money__absolute_sum_of_changes,money__longest_strike_below_mean,money__longest_strike_above_mean,money__count_above_mean,money__count_below_mean,money__last_location_of_maximum,money__first_location_of_maximum,money__last_location_of_minimum,money__first_location_of_minimum,money__percentage_of_reoccurring_values_to_all_values,money__percentage_of_reoccurring_datapoints_to_all_datapoints,money__sum_of_reoccurring_values,money__sum_of_reoccurring_data_points,money__ratio_value_number_to_time_series_length,money__maximum,money__minimum,money__time_reversal_asymmetry_statistic__lag_1,money__time_reversal_asymmetry_statistic__lag_2,money__time_reversal_asymmetry_statistic__lag_3,money__c3__lag_1,money__c3__lag_2,money__c3__lag_3,money__cid_ce__normalize_True,money__cid_ce__normalize_False,money__symmetry_looking__r_0.0,money__symmetry_looking__r_0.05,money__symmetry_looking__r_0.1,money__symmetry_looking__r_0.15000000000000002,money__symmetry_looking__r_0.2,money__symmetry_looking__r_0.25,money__symmetry_looking__r_0.30000000000000004,money__symmetry_looking__r_0.35000000000000003,money__symmetry_looking__r_0.4,money__symmetry_looking__r_0.45,money__symmetry_looking__r_0.5,money__symmetry_looking__r_0.55,money__symmetry_looking__r_0.6000000000000001,money__symmetry_looking__r_0.65,money__symmetry_looking__r_0.7000000000000001,money__symmetry_looking__r_0.75,money__symmetry_looking__r_0.8,money__symmetry_looking__r_0.8500000000000001,money__symmetry_looking__r_0.9,money__symmetry_looking__r_0.9500000000000001,money__large_standard_deviation__r_0.05,money__large_standard_deviation__r_0.1,money__large_standard_deviation__r_0.15000000000000002,money__large_standard_deviation__r_0.2,money__large_standard_deviation__r_0.25,money__large_standard_deviation__r_0.30000000000000004,money__large_standard_deviation__r_0.35000000000000003,money__large_standard_deviation__r_0.4,money__large_standard_deviation__r_0.45,money__large_standard_deviation__r_0.5,money__large_standard_deviation__r_0.55,money__large_standard_deviation__r_0.6000000000000001,money__large_standard_deviation__r_0.65,money__large_standard_deviation__r_0.7000000000000001,money__large_standard_deviation__r_0.75,money__large_standard_deviation__r_0.8,money__large_standard_deviation__r_0.8500000000000001,money__large_standard_deviation__r_0.9,money__large_standard_deviation__r_0.9500000000000001,money__quantile__q_0.1,money__quantile__q_0.2,money__quantile__q_0.3,money__quantile__q_0.4,money__quantile__q_0.6,money__quantile__q_0.7,money__quantile__q_0.8,money__quantile__q_0.9,money__autocorrelation__lag_0,money__autocorrelation__lag_1,money__autocorrelation__lag_2,money__autocorrelation__lag_3,money__autocorrelation__lag_4,"money__agg_autocorrelation__f_agg_""mean""__maxlag_40","money__agg_autocorrelation__f_agg_""median""__maxlag_40","money__agg_autocorrelation__f_agg_""var""__maxlag_40",money__partial_autocorrelation__lag_0,money__partial_autocorrelation__lag_1,money__number_cwt_peaks__n_1,money__number_cwt_peaks__n_5,money__number_peaks__n_1,money__number_peaks__n_3,money__number_peaks__n_5,money__number_peaks__n_10,money__number_peaks__n_50,money__binned_entropy__max_bins_10,money__index_mass_quantile__q_0.1,money__index_mass_quantile__q_0.2,money__index_mass_quantile__q_0.3,money__index_mass_quantile__q_0.4,money__index_mass_quantile__q_0.6,money__index_mass_quantile__q_0.7,money__index_mass_quantile__q_0.8,money__index_mass_quantile__q_0.9,"money__cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)","money__cwt_coefficients__coeff_0__w_5__widths_(2, 5, 10, 20)","money__cwt_c

In [28]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37343 entries, 0 to 37342
Data columns (total 26 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   37343 non-null  object 
 1   Deposit              37343 non-null  int64  
 2   UpsellDate           37343 non-null  float64
 3   AccessoryRate        37343 non-null  float64
 4   rateTypeEntity       37343 non-null  int64  
 5   RatePerUnit          37343 non-null  int64  
 6   DaysOnDeposit        37343 non-null  int64  
 7   MainApplicantGender  37343 non-null  int64  
 8   Age                  37343 non-null  float64
 9   Region               37343 non-null  int64  
 10  Town                 37343 non-null  int64  
 11  Occupation           37343 non-null  int64  
 12  TotalContractValue   37343 non-null  float64
 13  ExpectedTermDate     37343 non-null  float64
 14  LastPaymentDate      37343 non-null  float64
 15  TransactionDates     37343 non-null 

In [29]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37343 entries, 0 to 37342
Columns: 269 entries, ID to money__permutation_entropy__dimension_5__tau_1
dtypes: float64(268), object(1)
memory usage: 76.6+ MB


In [30]:
i=0

for element in data1['PaymentsHistory']:
    element=element[1:]
    element=element[:-1]
    element = element.split (",")
    element=np.array(element)
    element=element.astype(np.float)
    data1.at[i,'PaymentsHistory']=element
    i=i+1
    
i=0

for element in data1['TransactionDates']:
    element=element[1:]
    element=element[:-1]
    element = element.split (",")
    element=np.array(element)
    element=element.astype(np.float)
    data1.at[i,'TransactionDates']=element
    i=i+1
    


In [31]:
da = pd.DataFrame([pd.Series(x) for x in data1.TransactionDates])
da.columns = ['TransDt_{}'.format(x+1) for x in da.columns]

da1 = pd.DataFrame([pd.Series(x) for x in data1.PaymentsHistory])
da1.columns = ['PayHis_{}'.format(x+1) for x in da1.columns]

dfm = pd.DataFrame(data1, columns = ['m1', 'm2','m3', 'm4', 'm5', 'm6'])

da2 = pd.concat([da,da1],axis=1)

data1.drop('m1', inplace=True, axis=1)
data1.drop('m2', inplace=True, axis=1)
data1.drop('m3', inplace=True, axis=1)
data1.drop('m4', inplace=True, axis=1)
data1.drop('m5', inplace=True, axis=1)
data1.drop('m6', inplace=True, axis=1)

data1= pd.concat([data1,da2],axis=1)
data1= pd.concat([data1,dfm],axis=1)

data1.drop('TransactionDates', inplace=True, axis=1)
data1.drop('PaymentsHistory', inplace=True, axis=1)

data1=data1.fillna(0)

data1.head()

,ID,Deposit,UpsellDate,AccessoryRate,rateTypeEntity,RatePerUnit,DaysOnDeposit,MainApplicantGender,Age,Region,Town,Occupation,TotalContractValue,ExpectedTermDate,LastPaymentDate,time,PaidAmount,PeriodDiff,TransDt_1,TransDt_2,TransDt_3,TransDt_4,TransDt_5,TransDt_6,TransDt_7,TransDt_8,TransDt_9,TransDt_10,TransDt_11,TransDt_12,TransDt_13,TransDt_14,TransDt_15,TransDt_16,TransDt_17,TransDt_18,TransDt_19,TransDt_20,TransDt_21,TransDt_22,TransDt_23,TransDt_24,TransDt_25,TransDt_26,TransDt_27,TransDt_28,TransDt_29,TransDt_30,TransDt_31,TransDt_32,TransDt_33,TransDt_34,TransDt_35,TransDt_36,TransDt_37,TransDt_38,TransDt_39,TransDt_40,TransDt_41,TransDt_42,TransDt_43,TransDt_44,TransDt_45,TransDt_46,TransDt_47,TransDt_48,TransDt_49,TransDt_50,TransDt_51,TransDt_52,PayHis_1,PayHis_2,PayHis_3,PayHis_4,PayHis_5,PayHis_6,PayHis_7,PayHis_8,PayHis_9,PayHis_10,PayHis_11,PayHis_12,PayHis_13,PayHis_14,PayHis_15,PayHis_16,PayHis_17,PayHis_18,PayHis_19,PayHis_20,PayHis_21,PayHis_22,PayHis_23,PayHis_24,PayHis_25,PayHis_26,PayHis_27,PayHis_28,PayHis_29,PayHis_30,PayHis_31,PayHis_32,PayHis_33,PayHis_34,PayHis_35,PayHis_36,PayHis_37,PayHis_38,PayHis_39,PayHis_40,PayHis_41,PayHis_42,PayHis_43,PayHis_44,PayHis_45,PayHis_46,PayHis_47,PayHis_48,PayHis_49,PayHis_50,PayHis_51,PayHis_52,m1,m2,m3,m4,m5,m6
0,ID_K00S4N4,2000,0.000000,0.0,0,35,7,1,41.0,1,5,5,14740.0,0.923077,0.769231,13.0,0.482361,0.461538,0.0,0.076923,0.153846,0.230769,0.307692,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.206920,0.071235,0.061737,0.071235,0.071235,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.083107,0.071235,0.080733,0.035617,0.118725,0.128223
1,ID_6L67PAA,2000,0.000000,0.0,0,35,7,1,33.0,0,13,5,14740.0,0.923077,4.076923,13.0,0.583446,3.538462,0.0,0.076923,0.153846,0.230769,0.384615,0.538462,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.271370,0.071235,0.071235,0.071235,0.071235,0.027137,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,ID_102CV85,2000,1.928571,35.0,0,35,7,0,48.0,2,22,0,29480.0,0.928571,1.000000,14.0,0.391113,0.428571,0.0,0.071429,0.142857,0.214286,0.285714,0.357143,0.428571,0.500000,0.571429,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.143996,0.033243,0.024932,0.024932,0.049864,0.024932,0.033243,0.031038,0.024932,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.033243,0.033243,0.041554,0.033243,0.031716,0.012042
3,ID_HXBJFHB,2000,0.000000,0.0,0,35,7,0,43.0,5,43,6,14740.0,0.923077,1.384615,13.0,0.751018,0.538462,0.0,0.076923,0.153846,0.230769,0.307692,0.384615,0.461538,0.538462,0.615385,0.692308,0.846154,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.152307,0.066486,0.066486,0.083107,0.066486,0.066486,0.066486,0.083107,0.049864,0.033243,0.016961,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.016961,0.067

In [32]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37343 entries, 0 to 37342
Columns: 128 entries, ID to m6
dtypes: float64(119), int64(8), object(1)
memory usage: 36.5+ MB


In [33]:
merged=pd.merge(data2,data1)

merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37343 entries, 0 to 37342
Columns: 396 entries, ID to m6
dtypes: float64(387), int64(8), object(1)
memory usage: 113.1+ MB


In [34]:
merged.to_csv("Final3.csv")

In [35]:
mer=pd.read_csv("Final3.csv")

In [36]:
mer.info()

mer.rename(columns={ mer.columns[0]: "A" }, inplace = True)  
mer.drop('A', inplace=True, axis=1)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37343 entries, 0 to 37342
Columns: 397 entries, Unnamed: 0 to m6
dtypes: float64(387), int64(9), object(1)
memory usage: 113.1+ MB


import re
mer = mer.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [37]:
mer.head()

,ID,money__variance_larger_than_standard_deviation,money__has_duplicate_max,money__has_duplicate_min,money__has_duplicate,money__sum_values,money__abs_energy,money__mean_abs_change,money__mean_change,money__mean_second_derivative_central,money__median,money__mean,money__length,money__standard_deviation,money__variation_coefficient,money__variance,money__skewness,money__kurtosis,money__root_mean_square,money__absolute_sum_of_changes,money__longest_strike_below_mean,money__longest_strike_above_mean,money__count_above_mean,money__count_below_mean,money__last_location_of_maximum,money__first_location_of_maximum,money__last_location_of_minimum,money__first_location_of_minimum,money__percentage_of_reoccurring_values_to_all_values,money__percentage_of_reoccurring_datapoints_to_all_datapoints,money__sum_of_reoccurring_values,money__sum_of_reoccurring_data_points,money__ratio_value_number_to_time_series_length,money__maximum,money__minimum,money__time_reversal_asymmetry_statistic__lag_1,money__time_reversal_asymmetry_statistic__lag_2,money__time_reversal_asymmetry_statistic__lag_3,money__c3__lag_1,money__c3__lag_2,money__c3__lag_3,money__cid_ce__normalize_True,money__cid_ce__normalize_False,money__symmetry_looking__r_0.0,money__symmetry_looking__r_0.05,money__symmetry_looking__r_0.1,money__symmetry_looking__r_0.15000000000000002,money__symmetry_looking__r_0.2,money__symmetry_looking__r_0.25,money__symmetry_looking__r_0.30000000000000004,money__symmetry_looking__r_0.35000000000000003,money__symmetry_looking__r_0.4,money__symmetry_looking__r_0.45,money__symmetry_looking__r_0.5,money__symmetry_looking__r_0.55,money__symmetry_looking__r_0.6000000000000001,money__symmetry_looking__r_0.65,money__symmetry_looking__r_0.7000000000000001,money__symmetry_looking__r_0.75,money__symmetry_looking__r_0.8,money__symmetry_looking__r_0.8500000000000001,money__symmetry_looking__r_0.9,money__symmetry_looking__r_0.9500000000000001,money__large_standard_deviation__r_0.05,money__large_standard_deviation__r_0.1,money__large_standard_deviation__r_0.15000000000000002,money__large_standard_deviation__r_0.2,money__large_standard_deviation__r_0.25,money__large_standard_deviation__r_0.30000000000000004,money__large_standard_deviation__r_0.35000000000000003,money__large_standard_deviation__r_0.4,money__large_standard_deviation__r_0.45,money__large_standard_deviation__r_0.5,money__large_standard_deviation__r_0.55,money__large_standard_deviation__r_0.6000000000000001,money__large_standard_deviation__r_0.65,money__large_standard_deviation__r_0.7000000000000001,money__large_standard_deviation__r_0.75,money__large_standard_deviation__r_0.8,money__large_standard_deviation__r_0.8500000000000001,money__large_standard_deviation__r_0.9,money__large_standard_deviation__r_0.9500000000000001,money__quantile__q_0.1,money__quantile__q_0.2,money__quantile__q_0.3,money__quantile__q_0.4,money__quantile__q_0.6,money__quantile__q_0.7,money__quantile__q_0.8,money__quantile__q_0.9,money__autocorrelation__lag_0,money__autocorrelation__lag_1,money__autocorrelation__lag_2,money__autocorrelation__lag_3,money__autocorrelation__lag_4,"money__agg_autocorrelation__f_agg_""mean""__maxlag_40","money__agg_autocorrelation__f_agg_""median""__maxlag_40","money__agg_autocorrelation__f_agg_""var""__maxlag_40",money__partial_autocorrelation__lag_0,money__partial_autocorrelation__lag_1,money__number_cwt_peaks__n_1,money__number_cwt_peaks__n_5,money__number_peaks__n_1,money__number_peaks__n_3,money__number_peaks__n_5,money__number_peaks__n_10,money__number_peaks__n_50,money__binned_entropy__max_bins_10,money__index_mass_quantile__q_0.1,money__index_mass_quantile__q_0.2,money__index_mass_quantile__q_0.3,money__index_mass_quantile__q_0.4,money__index_mass_quantile__q_0.6,money__index_mass_quantile__q_0.7,money__index_mass_quantile__q_0.8,money__index_mass_quantile__q_0.9,"money__cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)","money__cwt_coefficients__coeff_0__w_5__widths_(2, 5, 10, 20)","money__cwt_c

In [38]:
mask=mer['m1']==0

test=mer[mask]

train=mer[~mask]

In [39]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9336 entries, 0 to 37333
Columns: 396 entries, ID to m6
dtypes: float64(387), int64(8), object(1)
memory usage: 28.3+ MB


In [40]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28007 entries, 1 to 37342
Columns: 396 entries, ID to m6
dtypes: float64(387), int64(8), object(1)
memory usage: 84.8+ MB


In [41]:
X=mer.iloc[:,:-6]

X.drop('ID', inplace=True, axis=1)

y1=mer['m1']
y2=mer['m2']
y3=mer['m3']
y4=mer['m4']
y5=mer['m5']
y6=mer['m6']

X_train,X_test,y1_train,y1_test=train_test_split(X,y1,train_size=0.9,random_state=1)
X_train,X_test,y2_train,y2_test=train_test_split(X,y2,train_size=0.9,random_state=1)
X_train,X_test,y3_train,y3_test=train_test_split(X,y3,train_size=0.9,random_state=1)
X_train,X_test,y4_train,y4_test=train_test_split(X,y4,train_size=0.9,random_state=1)
X_train,X_test,y5_train,y5_test=train_test_split(X,y5,train_size=0.9,random_state=1)
X_train,X_test,y6_train,y6_test=train_test_split(X,y6,train_size=0.9,random_state=1)

X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37343 entries, 0 to 37342
Columns: 389 entries, money__variance_larger_than_standard_deviation to PayHis_52
dtypes: float64(381), int64(8)
memory usage: 110.8 MB


In [42]:
import sklearn
import lightgbm as lgb
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error

l1=LGBMRegressor()
l1.fit(X_train.to_numpy(),y1_train)

p=l1.predict(X_test.to_numpy())

mse = sklearn.metrics.mean_squared_error(y1_test, p)

print(mse)

l2=LGBMRegressor()
l2.fit(X_train.to_numpy(),y2_train)

p=l2.predict(X_test)

mse = sklearn.metrics.mean_squared_error(y2_test, p)

print(mse)

l3=LGBMRegressor()
l3.fit(X_train.to_numpy(),y3_train)

p=l3.predict(X_test.to_numpy())

mse = sklearn.metrics.mean_squared_error(y3_test, p)

print(mse)

l4=LGBMRegressor()
l4.fit(X_train.to_numpy(),y4_train)

p=l4.predict(X_test.to_numpy())

mse = sklearn.metrics.mean_squared_error(y4_test, p)

print(mse)

l5=LGBMRegressor()
l5.fit(X_train.to_numpy(),y5_train)

p=l5.predict(X_test.to_numpy())

mse = sklearn.metrics.mean_squared_error(y5_test, p)

print(mse)

l6=LGBMRegressor()
l6.fit(X_train.to_numpy(),y6_train)

p=l6.predict(X_test.to_numpy())

mse = sklearn.metrics.mean_squared_error(y6_test, p)

print(mse)

0.0006106195357721633
0.0006116577077287387
0.0007203618279203524
0.0009055794221277191
0.0007305638854933515
0.0021111069977631887


In [43]:
test.info()

test.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9336 entries, 0 to 37333
Columns: 396 entries, ID to m6
dtypes: float64(387), int64(8), object(1)
memory usage: 28.3+ MB


,ID,money__variance_larger_than_standard_deviation,money__has_duplicate_max,money__has_duplicate_min,money__has_duplicate,money__sum_values,money__abs_energy,money__mean_abs_change,money__mean_change,money__mean_second_derivative_central,money__median,money__mean,money__length,money__standard_deviation,money__variation_coefficient,money__variance,money__skewness,money__kurtosis,money__root_mean_square,money__absolute_sum_of_changes,money__longest_strike_below_mean,money__longest_strike_above_mean,money__count_above_mean,money__count_below_mean,money__last_location_of_maximum,money__first_location_of_maximum,money__last_location_of_minimum,money__first_location_of_minimum,money__percentage_of_reoccurring_values_to_all_values,money__percentage_of_reoccurring_datapoints_to_all_datapoints,money__sum_of_reoccurring_values,money__sum_of_reoccurring_data_points,money__ratio_value_number_to_time_series_length,money__maximum,money__minimum,money__time_reversal_asymmetry_statistic__lag_1,money__time_reversal_asymmetry_statistic__lag_2,money__time_reversal_asymmetry_statistic__lag_3,money__c3__lag_1,money__c3__lag_2,money__c3__lag_3,money__cid_ce__normalize_True,money__cid_ce__normalize_False,money__symmetry_looking__r_0.0,money__symmetry_looking__r_0.05,money__symmetry_looking__r_0.1,money__symmetry_looking__r_0.15000000000000002,money__symmetry_looking__r_0.2,money__symmetry_looking__r_0.25,money__symmetry_looking__r_0.30000000000000004,money__symmetry_looking__r_0.35000000000000003,money__symmetry_looking__r_0.4,money__symmetry_looking__r_0.45,money__symmetry_looking__r_0.5,money__symmetry_looking__r_0.55,money__symmetry_looking__r_0.6000000000000001,money__symmetry_looking__r_0.65,money__symmetry_looking__r_0.7000000000000001,money__symmetry_looking__r_0.75,money__symmetry_looking__r_0.8,money__symmetry_looking__r_0.8500000000000001,money__symmetry_looking__r_0.9,money__symmetry_looking__r_0.9500000000000001,money__large_standard_deviation__r_0.05,money__large_standard_deviation__r_0.1,money__large_standard_deviation__r_0.15000000000000002,money__large_standard_deviation__r_0.2,money__large_standard_deviation__r_0.25,money__large_standard_deviation__r_0.30000000000000004,money__large_standard_deviation__r_0.35000000000000003,money__large_standard_deviation__r_0.4,money__large_standard_deviation__r_0.45,money__large_standard_deviation__r_0.5,money__large_standard_deviation__r_0.55,money__large_standard_deviation__r_0.6000000000000001,money__large_standard_deviation__r_0.65,money__large_standard_deviation__r_0.7000000000000001,money__large_standard_deviation__r_0.75,money__large_standard_deviation__r_0.8,money__large_standard_deviation__r_0.8500000000000001,money__large_standard_deviation__r_0.9,money__large_standard_deviation__r_0.9500000000000001,money__quantile__q_0.1,money__quantile__q_0.2,money__quantile__q_0.3,money__quantile__q_0.4,money__quantile__q_0.6,money__quantile__q_0.7,money__quantile__q_0.8,money__quantile__q_0.9,money__autocorrelation__lag_0,money__autocorrelation__lag_1,money__autocorrelation__lag_2,money__autocorrelation__lag_3,money__autocorrelation__lag_4,"money__agg_autocorrelation__f_agg_""mean""__maxlag_40","money__agg_autocorrelation__f_agg_""median""__maxlag_40","money__agg_autocorrelation__f_agg_""var""__maxlag_40",money__partial_autocorrelation__lag_0,money__partial_autocorrelation__lag_1,money__number_cwt_peaks__n_1,money__number_cwt_peaks__n_5,money__number_peaks__n_1,money__number_peaks__n_3,money__number_peaks__n_5,money__number_peaks__n_10,money__number_peaks__n_50,money__binned_entropy__max_bins_10,money__index_mass_quantile__q_0.1,money__index_mass_quantile__q_0.2,money__index_mass_quantile__q_0.3,money__index_mass_quantile__q_0.4,money__index_mass_quantile__q_0.6,money__index_mass_quantile__q_0.7,money__index_mass_quantile__q_0.8,money__index_mass_quantile__q_0.9,"money__cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)","money__cwt_coefficients__coeff_0__w_5__widths_(2, 5, 10, 20)","money__cwt_c

In [44]:
order=pd.read_csv("test.csv")

In [45]:
order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9336 entries, 0 to 9335
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID                9336 non-null   object
 1   TransactionDates  9336 non-null   object
 2   PaymentsHistory   9336 non-null   object
dtypes: object(3)
memory usage: 218.9+ KB


In [46]:
order.drop('TransactionDates', inplace=True, axis=1)
order.drop('PaymentsHistory', inplace=True, axis=1)

test=pd.merge(order,test)

In [47]:
test.head()

,ID,money__variance_larger_than_standard_deviation,money__has_duplicate_max,money__has_duplicate_min,money__has_duplicate,money__sum_values,money__abs_energy,money__mean_abs_change,money__mean_change,money__mean_second_derivative_central,money__median,money__mean,money__length,money__standard_deviation,money__variation_coefficient,money__variance,money__skewness,money__kurtosis,money__root_mean_square,money__absolute_sum_of_changes,money__longest_strike_below_mean,money__longest_strike_above_mean,money__count_above_mean,money__count_below_mean,money__last_location_of_maximum,money__first_location_of_maximum,money__last_location_of_minimum,money__first_location_of_minimum,money__percentage_of_reoccurring_values_to_all_values,money__percentage_of_reoccurring_datapoints_to_all_datapoints,money__sum_of_reoccurring_values,money__sum_of_reoccurring_data_points,money__ratio_value_number_to_time_series_length,money__maximum,money__minimum,money__time_reversal_asymmetry_statistic__lag_1,money__time_reversal_asymmetry_statistic__lag_2,money__time_reversal_asymmetry_statistic__lag_3,money__c3__lag_1,money__c3__lag_2,money__c3__lag_3,money__cid_ce__normalize_True,money__cid_ce__normalize_False,money__symmetry_looking__r_0.0,money__symmetry_looking__r_0.05,money__symmetry_looking__r_0.1,money__symmetry_looking__r_0.15000000000000002,money__symmetry_looking__r_0.2,money__symmetry_looking__r_0.25,money__symmetry_looking__r_0.30000000000000004,money__symmetry_looking__r_0.35000000000000003,money__symmetry_looking__r_0.4,money__symmetry_looking__r_0.45,money__symmetry_looking__r_0.5,money__symmetry_looking__r_0.55,money__symmetry_looking__r_0.6000000000000001,money__symmetry_looking__r_0.65,money__symmetry_looking__r_0.7000000000000001,money__symmetry_looking__r_0.75,money__symmetry_looking__r_0.8,money__symmetry_looking__r_0.8500000000000001,money__symmetry_looking__r_0.9,money__symmetry_looking__r_0.9500000000000001,money__large_standard_deviation__r_0.05,money__large_standard_deviation__r_0.1,money__large_standard_deviation__r_0.15000000000000002,money__large_standard_deviation__r_0.2,money__large_standard_deviation__r_0.25,money__large_standard_deviation__r_0.30000000000000004,money__large_standard_deviation__r_0.35000000000000003,money__large_standard_deviation__r_0.4,money__large_standard_deviation__r_0.45,money__large_standard_deviation__r_0.5,money__large_standard_deviation__r_0.55,money__large_standard_deviation__r_0.6000000000000001,money__large_standard_deviation__r_0.65,money__large_standard_deviation__r_0.7000000000000001,money__large_standard_deviation__r_0.75,money__large_standard_deviation__r_0.8,money__large_standard_deviation__r_0.8500000000000001,money__large_standard_deviation__r_0.9,money__large_standard_deviation__r_0.9500000000000001,money__quantile__q_0.1,money__quantile__q_0.2,money__quantile__q_0.3,money__quantile__q_0.4,money__quantile__q_0.6,money__quantile__q_0.7,money__quantile__q_0.8,money__quantile__q_0.9,money__autocorrelation__lag_0,money__autocorrelation__lag_1,money__autocorrelation__lag_2,money__autocorrelation__lag_3,money__autocorrelation__lag_4,"money__agg_autocorrelation__f_agg_""mean""__maxlag_40","money__agg_autocorrelation__f_agg_""median""__maxlag_40","money__agg_autocorrelation__f_agg_""var""__maxlag_40",money__partial_autocorrelation__lag_0,money__partial_autocorrelation__lag_1,money__number_cwt_peaks__n_1,money__number_cwt_peaks__n_5,money__number_peaks__n_1,money__number_peaks__n_3,money__number_peaks__n_5,money__number_peaks__n_10,money__number_peaks__n_50,money__binned_entropy__max_bins_10,money__index_mass_quantile__q_0.1,money__index_mass_quantile__q_0.2,money__index_mass_quantile__q_0.3,money__index_mass_quantile__q_0.4,money__index_mass_quantile__q_0.6,money__index_mass_quantile__q_0.7,money__index_mass_quantile__q_0.8,money__index_mass_quantile__q_0.9,"money__cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)","money__cwt_coefficients__coeff_0__w_5__widths_(2, 5, 10, 20)","money__cwt_c

In [48]:
i_d_=test['ID']

test.drop('ID', inplace=True, axis=1)

test.drop('m1', inplace=True, axis=1)
test.drop('m2', inplace=True, axis=1)
test.drop('m3', inplace=True, axis=1)
test.drop('m4', inplace=True, axis=1)
test.drop('m5', inplace=True, axis=1)
test.drop('m6', inplace=True, axis=1)

In [49]:
p1=l1.predict(test)
p2=l2.predict(test)
p3=l3.predict(test)
p4=l4.predict(test)
p5=l5.predict(test)
p6=l6.predict(test)

pres=np.array([])

m=0

for x in test['TotalContractValue']:
    pres = np.append(pres, p1[m]*x)
    pres = np.append(pres, p2[m]*x)
    pres = np.append(pres, p3[m]*x)
    pres = np.append(pres, p4[m]*x)
    pres = np.append(pres, p5[m]*x)
    pres = np.append(pres, p6[m]*x)
    m=m+1
    
display(len(pres))

56016

In [50]:
ss.head(n=12)

ss.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56016 entries, 0 to 56015
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      56016 non-null  object 
 1   Target  56016 non-null  float64
dtypes: float64(1), object(1)
memory usage: 875.4+ KB


In [51]:
ss["Target"] = pres

In [52]:
ss.head(n=12)

,ID,Target
0,ID_6L67PAA x m1,271.041337
1,ID_6L67PAA x m2,362.438457
2,ID_6L67PAA x m3,290.503945
3,ID_6L67PAA x m4,608.571369
4,ID_6L67PAA x m5,303.762229
5,ID_6L67PAA x m6,297.229335
6,ID_VJ80SX2 x m1,656.803384
7,ID_VJ80SX2 x m2,704.797495
8,ID_VJ80SX2 x m3,657.417104
9,ID_VJ80SX2 x m4,690.620273


In [53]:
ss.to_csv('baseline_sub1.csv', index=False)